# Testing Topic Modelling

This has to be done on full dataset. Let's see if my computer can handle it. Else we will move this to Google Colab.  

I first try to base the topics on the tf-idf vectors.  

In [1]:
# general:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import time # will be use to choose the faster solution
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer # Normalize samples individually to unit norm.

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import sklearn
import pickle # to save models, for instance LDA outputs


# NLP:
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD # LSA
from sklearn.decomposition import NMF # NMF
from sklearn.decomposition import LatentDirichletAllocation # LDA


/usr/local/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/usr/local/lib/python3.7/site-packages/gensim/models/doc2vec.py:73: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import namedtuple, defaultdict, Iterable


In [2]:
# load data:
raw_winedata = pd.read_csv("../data/winemag-data-190314.csv").drop("Unnamed: 0", axis=1) # indexes were stored as col
initial_number_of_rows = raw_winedata.shape[0]
print(raw_winedata.shape)
raw_winedata.head()


(141617, 14)


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,vintage
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013.0
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011.0
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013.0
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013.0
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012.0


In [3]:
# try to keep only varieties that occur more than n times:
n = 500
variety_counts = raw_winedata.variety.value_counts()
winedata = raw_winedata[raw_winedata.variety.isin(variety_counts.index[variety_counts.gt(n)])] # pandas.DataFrame.gt = get greater
print(winedata.shape)


(122856, 14)


Even if we strongly limit the minimal number of occurences for a variety, we still get a large dataset, and computations will be slow on a laptop.

In [4]:
# try to keep only province that occur more than n times:
n = 500
province_counts = winedata.province.value_counts()
winedata = winedata[winedata.province.isin(province_counts.index[province_counts.gt(n)])] # pandas.DataFrame.gt = get greater
print(winedata.shape)

(106924, 14)


**We keep only the varieties and provinces that are occuring more than 500 times, because we want to try to predict those - in a parallel exercise as building the unsupervised learning system - using the text desciption.** *texte en italique*

In [5]:
domain_specific_stopwords = ['$', ' ', '’s', 'wine', 'winemaker', 'winemaking', 'winery', 
                            '2020–2030', '2–3', '3–4', '4–5', '5–6', '6–8', 
                             'château', 'village', 'beaujolais', 'domaine', 
                             'côte', 'saint', 'village', 'parcel', 'parcels',
                             'I', 'flavors',
                             'now-2015', 'now-2018', 'now-2025']
                        # consider:
#                             cabernet, cabernets, douro, nacional,  
#                              widely, muscat,
#                              willamette, bordeaux, pommard, rioja, barbaresco, chianti]

# a list of all wine varieties. this is not perfect as some varieties are compound words. But it's a start.
variety_stopwords = winedata.variety.unique().tolist()
variety_stopwords = [str(variety).lower() for variety in variety_stopwords] # convert to lowercase
variety_stopwords.append(['pinot', 'cabernet', 'cabernets', 'sauvignon', 'grigio', 'sirah' ]) # as frequent and only happens as compound word

# a list of all wine provinces. 
province_stopwords = winedata.province.unique().tolist()
province_stopwords = [str(province).lower() for province in province_stopwords] # convert to lowercase

In [6]:
# Updating our tokenizer function:
def tokenize_and_clean(description):

    # Tokenize:
    mytokens = nlp(description)

    # Removing stopwords, punctuation and convert to lower_case + AND NUMBERS (or use is_digit?)
    mytokens = [token.lower_ for token in mytokens if not token.is_punct and not token.is_stop and not token.like_num and not token.is_digit]
    
    # remove domain-specific stopwords:
    mytokens = [token for token in mytokens if token not in domain_specific_stopwords]
            
    # remove wine variety occurrences in the descriptions:
    mytokens = [token for token in mytokens if token not in variety_stopwords]
                
    # Return preprocessed list of tokens
    return mytokens

In [7]:
# We do not need word vectors here, so we can upload the small English model from spaCy:
nlp = spacy.load('en_core_web_sm', disable=['tagger', 'parser', 'ner'])

In [8]:
time0 = time.time()

tfidf_vector = TfidfVectorizer(tokenizer = tokenize_and_clean, # using our custom tokenizer
                               ngram_range=(1,1),
                               max_df=0.95, # ignore t that have a df higher than max_df (corpus-specific stopwords)
                               min_df=10, # ignore terms that have a doc freq lower than threshold.
                               max_features=2000
                            )
# Applying the vectorizer:
wine_tfidf = tfidf_vector.fit_transform(winedata.description) # input: the column "description"

# Getting the word list.
terms = tfidf_vector.get_feature_names()

print('Done! it took', time.time()-time0, 'seconds.')

Done! it took 23.535081386566162 seconds.


## Trying Topic Modelling

In [9]:
# Number of topics:
ntopics=5

# Number of words to look at for each topic.
n_top_words = 10

# to print the top n words:
topwords = pd.DataFrame(index=range(0,ntopics))


### LSA and NMF

In [10]:
# Linking words to topics
def word_topic(tfidf, solution, wordlist):
    
    # Loading scores for each word on each topic/component.
    words_by_topic = tfidf.T * solution

    # Linking the loadings to the words in an easy-to-read way.
    components = pd.DataFrame(words_by_topic, index=wordlist)
    
    return components

In [11]:
# Extracts the top N words and their loadings for each topic.
def top_words(components, n_top_words):
    n_topics = range(components.shape[1])
    
    index = np.repeat(n_topics, n_top_words, axis=0)
    topwords = pd.Series(index=index) # initiate a Series where to store the topwords of each topic
    fullist=[] # usual code doesn't work
    
    for column in n_topics:
        # Sort the column so that highest loadings are at the top.
        sortedwords = components.iloc[:,column].sort_values(ascending=False)
        # Choose the N highest loadings.
        chosen = sortedwords[:n_top_words]
        # Combine loading and index into a string.
        chosenlist = chosen.index + "  "+ round(chosen,2).map(str)
        
        fullist.append(chosenlist)
    
#     topwords = pd.Series(fullist, index=index)
        
#         topwords.loc[column] = chosenlist
    return(fullist) # (topwords)

In [12]:
# LSA

time0 = time.time()

svd = TruncatedSVD(ntopics)
lsa = make_pipeline(svd, Normalizer(copy=False))
wine_lsa = lsa.fit_transform(wine_tfidf)

components_lsa = word_topic(wine_tfidf, wine_lsa, terms)

topwords['LSA'] = top_words(components_lsa, n_top_words) 

print('Done! It took', time.time()-time0, 'seconds.')


Done! It took 0.5203042030334473 seconds.


In [13]:
# NNMF
time0 = time.time()

svd = TruncatedSVD(ntopics)
lsa = make_pipeline(svd, Normalizer(copy=False))
wine_lsa = lsa.fit_transform(wine_tfidf)

components_lsa = word_topic(wine_tfidf, wine_lsa, terms)


nmf = NMF(alpha=0.0, 
          init='nndsvdar', # how starting value are calculated
          l1_ratio=0.0, # Sets whether regularization is L2 (0), L1 (1), or a combination (values between 0 and 1)
          max_iter=200, # when to stop even if the model is not converging (to prevent running forever)
          n_components=ntopics, 
          random_state=0, 
          solver='cd', # Use Coordinate Descent to solve
          tol=0.0001, # model will stop if tfidf-WH <= tol
          verbose=0 # amount of output to give while iterating
         )
wine_nmf = nmf.fit_transform(wine_tfidf) 

components_nmf = word_topic(wine_tfidf, wine_nmf, terms)

topwords['NMF'] = top_words(components_nmf, n_top_words) 
    
print('Done! It took', time.time()-time0, 'seconds.')


Done! It took 2.4736430644989014 seconds.


In [14]:
for i in range(0,ntopics):
    print('topic', i, ':\nLSA:\n', topwords.LSA[i], '\nNMF:\n', topwords.NMF[i],'\n')


topic 0 :
LSA:
 fruit        fruit  3142.64
tannins    tannins  2481.28
acidity    acidity  2472.25
black        black  2386.86
aromas      aromas  2386.17
drink        drink  2374.27
cherry      cherry  2323.03
ripe          ripe  2271.61
palate      palate  2239.31
finish      finish  2155.67
dtype: object 
NMF:
 black            black  66.15
aromas           aromas  62.4
palate          palate  61.75
cherry          cherry  61.07
tannins        tannins  57.36
berry            berry  42.74
spice            spice  38.71
red                red  38.68
alongside    alongside  36.49
offers          offers  36.37
dtype: object 

topic 1 :
LSA:
 acidity        acidity  916.06
fruits          fruits  758.78
drink            drink  740.93
crisp            crisp  679.52
ripe              ripe  593.11
fruity          fruity  583.46
ready            ready  516.37
character    character  515.61
rich               rich  508.7
texture        texture  480.63
dtype: object 
NMF:
 drink       drink  1

### LDA

In [15]:
# LDA

time0 = time.time()

lda = LatentDirichletAllocation(n_components=ntopics, 
          doc_topic_prior=None, # Prior = 1/n_documents
          topic_word_prior=1/ntopics,
          learning_decay=0.7, # Convergence rate.
          learning_offset=10.0, # Causes earlier iterations to have less influence on the learning
          max_iter=10, # when to stop even if the model is not converging (to prevent running forever)
          evaluate_every=-1, # Do not evaluate perplexity, as it slows training time.
          mean_change_tol=0.001, # Stop updating the document topic distribution in the E-step when mean change is < tol
          max_doc_update_iter=100, # When to stop updating the document topic distribution in the E-step even if tol is not reached
          n_jobs=-1, # Use all available CPUs to speed up processing time.
          verbose=0, # amount of output to give while iterating
          random_state=0
         )
wine_lda = lda.fit(wine_tfidf)

print('Done! It took', time.time()-time0, 'seconds.')


Done! It took 93.41152429580688 seconds.


Print topics:


## pyLDAvis

A good topic model will have non-overlapping, fairly big sized blobs for each topic.

In [16]:
pyLDAvis.sklearn.prepare(wine_lda, wine_tfidf, tfidf_vector, mds='PCoA') # try also mds='tsne'

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.052983  0.108156       1        1  30.237771
1      0.201737 -0.075621       2        1  19.192348
0     -0.075042 -0.052466       3        1  18.324113
2      0.034918  0.133849       4        1  18.265324
3     -0.108631 -0.113918       5        1  13.980444, topic_info=     Category         Freq    Term        Total  loglift  logprob
744   Default  1957.000000  fruits  1957.000000  30.0000  30.0000
71    Default  1665.000000   apple  1665.000000  29.0000  29.0000
540   Default  3230.000000   drink  3230.000000  28.0000  28.0000
1426  Default  1050.000000   ready  1050.000000  27.0000  27.0000
1969  Default  1226.000000    wood  1226.000000  26.0000  26.0000
...       ...          ...     ...          ...      ...      ...
741    Topic5   477.331518   fruit  4039.400902  -0.1681  -4.8680
1210   Topic5   336.833616  offers  1560.475926   0.4344  -5.2166
566    Topic5   269.476985    easy   827.274157   0.8459  -5.4397
1483   Topic5   284.597450    ripe  3019.299303  -0.3942  -5.3851
184    Topic5   278.068938  bodied  1694.569669   0.1602  -5.4083

[410 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
13        1  0.131213  acidity
13        2  0.464223  acidity
13        3  0.156986  acidity
13        4  0.086987  acidity
13        5  0.160794  acidity
...     ...       ...      ...
1995      1  0.087858     zest
1995      2  0.219646     zest
1995      3  0.158982     zest
1995      5  0.533426     zest
1997      4  0.998168      zin

[901 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 1, 3, 4])

In [17]:
time0 = time.time()

# Log Likelyhood: Higher the better
print("Log Likelihood: ", wine_lda.score(wine_tfidf))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", wine_lda.perplexity(wine_tfidf))

print('Done! it took', time.time()-time0, 'sec.')

# See model parameters
wine_lda.get_params()



Log Likelihood:  -3230460.9748833817
Perplexity:  1443.614541532508
Done! it took 22.463308095932007 sec.


{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'batch',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 5,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 0,
 'topic_word_prior': 0.2,
 'total_samples': 1000000.0,
 'verbose': 0}

In [18]:

pkl_filename = "../data/pickle_LDA_10t.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(wine_lda, file)


In [19]:
# RUN THIS TO LOAD A MODEL:
# # Load from file
# import pickle

# pkl_filename = "../data/pickle_LDA_10t.pkl"
# with open(pkl_filename, 'rb') as file:
#     wine_lda = pickle.load(file)
    
# wine_lda

#### Try different number of topics


In [20]:
# Takes 15min !!
time0 = time.time()

all_ntopics = [4,5,6,7,8,20]

results = pd.DataFrame(all_ntopics, columns=['ntopics'], index=all_ntopics)

for ntopics in all_ntopics:
    lda = LatentDirichletAllocation(n_components=ntopics, 
          doc_topic_prior=None, # Prior = 1/n_documents
          topic_word_prior=1/ntopics,
          learning_decay=0.7, # Convergence rate.
          learning_offset=10.0, # Causes earlier iterations to have less influence on the learning
          max_iter=10, # when to stop even if the model is not converging (to prevent running forever)
          evaluate_every=-1, # Do not evaluate perplexity, as it slows training time.
          mean_change_tol=0.001, # Stop updating the document topic distribution in the E-step when mean change is < tol
          max_doc_update_iter=100, # When to stop updating the document topic distribution in the E-step even if tol is not reached
          n_jobs=-1, # Use all available CPUs to speed up processing time.
          verbose=0, # amount of output to give while iterating
          random_state=0
         )
    results.loc[ntopics, 'LDA'] = lda.fit(wine_tfidf)
    
    results.loc[ntopics, 'LL'] = results.loc[ntopics, 'LDA'].score(wine_tfidf)
    results.loc[ntopics, 'perplexity'] = results.loc[ntopics, 'LDA'].perplexity(wine_tfidf)

print('Done! It took', time.time()-time0, 'seconds.')


Done! It took 643.7981691360474 seconds.


In [21]:
# overview of results:
results

,ntopics,LDA,LL,perplexity
4,4,"LatentDirichletAllocation(batch_size=128, doc_...",-3.206136e+06,1366.662597
5,5,"LatentDirichletAllocation(batch_size=128, doc_...",-3.230461e+06,1443.614542
6,6,"LatentDirichletAllocation(batch_size=128, doc_...",-3.243728e+06,1487.397181
7,7,"LatentDirichletAllocation(batch_size=128, doc_...",-3.247429e+06,1499.843761
8,8,"LatentDirichletAllocation(batch_size=128, doc_...",-3.263258e+06,1554.273344
20,20,"LatentDirichletAllocation(batch_size=128, doc_...",-3.379549e+06,2019.583906


In [22]:
# SAVE ALL THESE OUTPUTS !!!

for ntopics in all_ntopics:
    # Save to file:
    pkl_filename = '../data/pickle_LDA_' + str(ntopics) + 't.pkl'
    with open(pkl_filename, 'wb') as file:
        pickle.dump(results.loc[ntopics, 'LDA'], file)

In [23]:
# 6 topics:
pyLDAvis.sklearn.prepare(results.loc[6, 'LDA'], wine_tfidf, tfidf_vector, mds='PCoA') # or mds='mmds'

# Save to file:
pkl_filename = "../data/pickle_LDA_10t.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(wine_lda, file)


/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [24]:
# 7 topics:
pyLDAvis.sklearn.prepare(results.loc[7, 'LDA'], wine_tfidf, tfidf_vector, mds='PCoA') # or mds='mmds'

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.029375 -0.028616       1        1  19.896787
1     -0.121301 -0.110460       2        1  17.619495
5      0.063559  0.006933       3        1  14.320309
0     -0.138230  0.078753       4        1  13.078054
3     -0.063696  0.169194       5        1  12.745123
2     -0.023212 -0.137401       6        1  11.999207
6      0.253505  0.021596       7        1  10.341025, topic_info=     Category         Freq       Term        Total  loglift  logprob
744   Default  1896.000000     fruits  1896.000000  30.0000  30.0000
1794  Default  3274.000000    tannins  3274.000000  29.0000  29.0000
71    Default  1614.000000      apple  1614.000000  28.0000  28.0000
49    Default   942.000000  alongside   942.000000  27.0000  27.0000
1248  Default  3114.000000     palate  3114.000000  26.0000  26.0000
...       ...          ...        ...          ...      ...      ...
1962   Topic7   285.976516       wild   691.474794   1.3861  -5.0787
858    Topic7   303.421464       hint   934.866749   1.1438  -5.0195
460    Topic7   319.546425       dark  1662.878943   0.6196  -4.9678
1321   Topic7   313.562093       plum  1766.949192   0.5400  -4.9867
843    Topic7   305.448457       herb  1061.870880   1.0230  -5.0129

[536 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4         4  0.109106   acacia
4         5  0.872848   acacia
13        1  0.075369  acidity
13        2  0.442345  acidity
13        3  0.064602  acidity
...     ...       ...      ...
1996      6  0.020686    zesty
1996      7  0.043671    zesty
1997      1  0.138759      zin
1997      3  0.035809      zin
1997      6  0.819126      zin

[1685 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 6, 1, 4, 3, 7])

In [25]:
# 8 topics:
pyLDAvis.sklearn.prepare(results.loc[8, 'LDA'], wine_tfidf, tfidf_vector, mds='PCoA') # or mds='mmds'

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.012364  0.040111       1        1  17.519819
1     -0.137313  0.111015       2        1  16.866671
5      0.048812  0.009350       3        1  13.719597
0     -0.129694 -0.096037       4        1  12.509237
2     -0.049605  0.139965       5        1  11.679659
7     -0.053521 -0.124282       6        1  11.427515
6      0.258041  0.029966       7        1   9.315599
3      0.050916 -0.110087       8        1   6.961902, topic_info=     Category         Freq       Term        Total  loglift  logprob
744   Default  1916.000000     fruits  1916.000000  30.0000  30.0000
49    Default   947.000000  alongside   947.000000  29.0000  29.0000
81    Default  3266.000000     aromas  3266.000000  28.0000  28.0000
1248  Default  3101.000000     palate  3101.000000  27.0000  27.0000
1794  Default  3263.000000    tannins  3263.000000  26.0000  26.0000
...       ...          ...        ...          ...      ...      ...
1189   Topic8   167.496033      notes  1939.035417   0.2157  -5.2180
13     Topic8   181.429192    acidity  3360.237530  -0.2542  -5.1381
169    Topic8   159.740531      blend  1600.020905   0.3605  -5.2654
437    Topic8   152.706860      crisp  1722.041268   0.2420  -5.3105
151    Topic8   152.233999      berry  2077.856386   0.0510  -5.3136

[596 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
4         4  0.110336      acacia
4         8  0.882685      acacia
13        1  0.056246     acidity
13        2  0.437469     acidity
13        3  0.061900     acidity
...     ...       ...         ...
1994      6  0.026026  youthfully
1994      7  0.936950  youthfully
1997      1  0.159919         zin
1997      3  0.051739         zin
1997      5  0.785485         zin

[2124 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 6, 1, 3, 8, 7, 4])

In [26]:
# 4 topics:
pyLDAvis.sklearn.prepare(results.loc[4, 'LDA'], wine_tfidf, tfidf_vector, mds='PCoA') # or mds='mmds'

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.070683 -0.114785       1        1  31.977261
0     -0.078564 -0.028932       2        1  25.422890
3     -0.068522  0.145598       3        1  21.408732
1      0.217769 -0.001881       4        1  21.191117, topic_info=     Category         Freq       Term        Total  loglift  logprob
744   Default  2019.000000     fruits  2019.000000  30.0000  30.0000
540   Default  3271.000000      drink  3271.000000  29.0000  29.0000
1426  Default  1081.000000      ready  1081.000000  28.0000  28.0000
34    Default  1038.000000      aging  1038.000000  27.0000  27.0000
303   Default  1352.000000  character  1352.000000  26.0000  26.0000
...       ...          ...        ...          ...      ...      ...
1431   Topic4   869.355985        red  2529.986393   0.4834  -4.6844
114    Topic4   595.876631   balanced  1322.085234   0.7547  -5.0621
735    Topic4   636.827558      fresh  1974.102252   0.4202  -4.9956
162    Topic4   709.257788      black  3005.739510   0.1075  -4.8879
549    Topic4   559.772989        dry  2178.920177   0.1925  -5.1246

[335 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4         3  0.982006   acacia
13        1  0.130556  acidity
13        2  0.110582  acidity
13        3  0.258796  acidity
13        4  0.499933  acidity
...     ...       ...      ...
1996      2  0.065060    zesty
1996      3  0.690979    zesty
1996      4  0.183962    zesty
1999      2  0.020871    zippy
1999      3  0.970484    zippy

[634 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2])

In [27]:
# 5 topics:
pyLDAvis.sklearn.prepare(results.loc[5, 'LDA'], wine_tfidf, tfidf_vector, mds='PCoA') # or mds='mmds'

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.052983  0.108156       1        1  30.237771
1      0.201737 -0.075621       2        1  19.192348
0     -0.075042 -0.052466       3        1  18.324113
2      0.034918  0.133849       4        1  18.265324
3     -0.108631 -0.113918       5        1  13.980444, topic_info=     Category         Freq    Term        Total  loglift  logprob
744   Default  1957.000000  fruits  1957.000000  30.0000  30.0000
71    Default  1665.000000   apple  1665.000000  29.0000  29.0000
540   Default  3230.000000   drink  3230.000000  28.0000  28.0000
1426  Default  1050.000000   ready  1050.000000  27.0000  27.0000
1969  Default  1226.000000    wood  1226.000000  26.0000  26.0000
...       ...          ...     ...          ...      ...      ...
741    Topic5   477.331518   fruit  4039.400902  -0.1681  -4.8680
1210   Topic5   336.833616  offers  1560.475926   0.4344  -5.2166
566    Topic5   269.476985    easy   827.274157   0.8459  -5.4397
1483   Topic5   284.597450    ripe  3019.299303  -0.3942  -5.3851
184    Topic5   278.068938  bodied  1694.569669   0.1602  -5.4083

[410 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
13        1  0.131213  acidity
13        2  0.464223  acidity
13        3  0.156986  acidity
13        4  0.086987  acidity
13        5  0.160794  acidity
...     ...       ...      ...
1995      1  0.087858     zest
1995      2  0.219646     zest
1995      3  0.158982     zest
1995      5  0.533426     zest
1997      4  0.998168      zin

[901 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 1, 3, 4])

In [28]:
# 20 topics:
pyLDAvis.sklearn.prepare(results.loc[20, 'LDA'], wine_tfidf, tfidf_vector, mds='PCoA') # or mds='mmds'

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2      0.024763  0.202014       1        1  8.598162
1     -0.158408  0.127438       2        1  8.353046
6      0.150259 -0.161444       3        1  7.392743
5      0.063494 -0.034972       4        1  6.197165
0     -0.180474 -0.054625       5        1  6.172477
16     0.034143 -0.092654       6        1  5.950116
13     0.024903  0.044317       7        1  5.748718
7     -0.150948 -0.078743       8        1  5.695941
17     0.062131  0.156247       9        1  5.131920
18    -0.019694  0.074041      10        1  4.887004
11     0.034389  0.022314      11        1  4.392682
14     0.065835  0.003435      12        1  4.209576
19     0.101282 -0.018767      13        1  4.090950
9      0.100852 -0.085428      14        1  3.926410
10    -0.176288  0.088447      15        1  3.789588
3     -0.187086 -0.162270      16        1  3.514454
12    -0.069278 -0.044504      17        1  3.230594
4      0.085853 -0.012232      18        1  3.150481
15     0.143178  0.054883      19        1  2.967083
8      0.051094 -0.027499      20        1  2.600888, topic_info=     Category         Freq       Term        Total  loglift  logprob
257   Default  1646.000000   cabernet  1646.000000  30.0000  30.0000
81    Default  3191.000000     aromas  3191.000000  29.0000  29.0000
1542  Default  1178.000000  sauvignon  1178.000000  28.0000  28.0000
71    Default  1562.000000      apple  1562.000000  27.0000  27.0000
733   Default   605.000000      franc   605.000000  26.0000  26.0000
...       ...          ...        ...          ...      ...      ...
1830  Topic20    61.681238      tight   848.620431   1.0277  -5.2324
1189  Topic20    68.640135      notes  1959.684029   0.2977  -5.1255
1210  Topic20    64.576906     offers  1593.832175   0.4433  -5.1865
538   Topic20    62.237049      dried  1192.798339   0.6962  -5.2234
1985  Topic20    61.593860      years  1137.678158   0.7331  -5.2338

[1255 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3         5  0.078338  abundant
3         7  0.019585  abundant
3         8  0.048962  abundant
3        10  0.019585  abundant
3        11  0.019585  abundant
...     ...       ...       ...
1997     13  0.009295       zin
1997     17  0.013942       zin
1997     18  0.004647       zin
1997     19  0.055770       zin
1997     20  0.004647       zin

[9195 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 7, 6, 1, 17, 14, 8, 18, 19, 12, 15, 20, 10, 11, 4, 13, 5, 16, 9])

## Build Features from Topics

## Similarity Matrix